<b>Cellular automaton</b> is a discrete system of computation developed in the early days of computer.<br>
It is part of [automata theory](https://en.wikipedia.org/wiki/Automata_theory) and has many applications in form of simulation on various areas like physics and biology<br>
You can read more about Cellular automaton [here](https://en.wikipedia.org/wiki/Cellular_automaton)<br>

Here we will go through the process of training different models to predict next generation (state)<br>

 <ul>
  <li>Develop python class representing Cellular automaton</li>
  <li>Training Conway's Game of life Pytorch model</li>
  <li>Trainig Newral Network multiple rules model to predict next state according predefined rules</li>
  <li>Classifying rule class base on 2 consequent states</li>
</ul> 

In [1]:
import numpy as np
from itertools import count
from collections import namedtuple

from IPython import display
from IPython.display import HTML

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pylab as plt
import matplotlib.animation as animation
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

import time
from inspect import signature

The CellAuto is a class that apply rules to generate the next generation.<br>
The Next generation matrix is calculated base on each cell neighbours sum.<br>

The Rule is base on 2 list parameters <b>live, dead</b> .<br>
<b>live</b> list - if current cell is live (e.g its value = 1) and the sum of its 8 neighbours is in the list the cell will kipp living otherwise will dead<br>
<b>dead</b> list - if current cell is dead (e.g its value = 0) and the sum of its 8 neighbours is in the list the cell will kipp living otherwise will dead<br>

![from wikipedia](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Moore_neighborhood_with_cardinal_directions.svg/220px-Moore_neighborhood_with_cardinal_directions.svg.png)


### CellAuto class

In [2]:
W = H = 16 # height, width

class CellAuto:    
    def __init__(self, live: list, dead: list, distribution: list=[.5, .5]):
        self.live = live # num of neighbours when cell is alive in order to keep alive
        self.dead = dead # num of neighbours when cell is dead in order cell to be alive again
        self.data = np.random.choice(2, H*W, p=distribution).astype(np.uint8).reshape((H, W))
        
    def get_live_neighbours(self, i, c):
        sum = 0
        for x in range(max(0, i-1), min(H, i+2)):
            for m in range(max(0, c-1), min(W, c+2)):
                sum += self.data[x, m]
        return sum - self.data[i, c]
        
    def tick(self):
        """ Calculates next generation """
        arr = np.zeros_like(self.data)
        for i in range(H):
            for c in range(W):
                neighbours = self.get_live_neighbours(i, c)
                if self.data[i, c] > 0 and neighbours in self.live:
                    arr[i, c] = 1
                elif self.data[i, c] == 0 and neighbours in self.dead:
                    arr[i, c] = 1
        self.data[:, :] = arr[:, :]
        
    def __str__(self):
        s = {"live": str(self.live), "dead": str(self.dead)}
        return str(s).replace("'", "").replace(" ", "")


<b>Animator</b> is a class to animate results of CellAuto.<br>
It has the ability to run N generations with or without NN model.<br>
In case no model supplies only calculated results will animated<br>
otherwise the animation consists of both calculated (real) animation alongside the predicted (by the model).<br>

### Animator class

In [42]:
class Animator:
    def __init__(self, auto_cell, model=None, cmap='cividis'):
        self.cellula = auto_cell
        self.model = model
        self.cmap = cmap
        sig = str(self.cellula)
        assert sig in RULES_MAP, "rule must be part from training set rules"
        self.rule = RULES_MAP[sig]
        
    def prepare_data(self, n_iterations):
        real, predicted = [], []
        real.append(self.cellula.data.copy())
        if self.model:
            predicted.append(self.cellula.data.copy())
    
        for i in range(n_iterations):
            if self.model:
                predicted.append(self.predict())
            self.cellula.tick()
            real.append(self.cellula.data.copy())
        return real, predicted
    
    def predict(self):
        data = self.cellula.data
        arr = data.astype(np.float32).reshape((1, 1, W, H))
        n_params = len(signature(self.model.__init__).parameters)
        
        if n_params == 1:
            res = self.model(torch.from_numpy(arr))
        elif n_params == 2:
            res = self.model(torch.from_numpy(arr), self.rule)
        res[res<.5] = 0
        res[res>=.5] = 1
        return res.detach().numpy().reshape((H, W))
    
    def animate_single(self, data, figsize=(5, 5), interval=1000):
        ims = []
        fig = plt.figure(figsize=figsize)
        for plane in data:
            im = plt.imshow(plane, animated=True, cmap=self.cmap)
            plt.title(str(self.cellula))
            plt.axis(False)
            ims.append([im])
    
        anim = animation.ArtistAnimation(fig, ims, interval=interval, blit=True,
                                repeat_delay=interval)
        plt.close(anim._fig)
        return anim
    
    def animate_double(self, data1, data2, padding=1, figsize=(8, 4), interval=1000):
        plane = np.zeros((H, W*2 + padding), dtype=np.uint8)
        plane[:, :] += 2
        ims = []
        fig = plt.figure(figsize=figsize)
        for a, b in zip(data1, data2):
            plane[:, :W] = a
            plane[:, W + padding:] = b
            plt.title(f"REAL                                                 PREDICTED")
            im = plt.imshow(plane, animated=True, cmap=self.cmap)
            plt.axis(False)
            ims.append([im])
    
        anim = animation.ArtistAnimation(fig, ims, interval=interval, blit=True,
                                repeat_delay=interval)
        plt.close(anim._fig)
        return anim
    
    def __call__(self, n_iterations=32, figsize=(8, 4), path=None, interval=1000):
        real, predicted = self.prepare_data(n_iterations)
        if predicted:
            anim = self.animate_double(real, predicted, figsize=figsize, interval=interval)
        else:
            anim = self.animate_single(real, figsize=figsize, interval=interval)
        if path:
            anim.save(path)
        return anim
    

In [33]:
# the actual NN rule class
rules = torch.eye(3) 

# mapping CellAuto rules to the NN rule class
RULES_MAP = {
    str(CellAuto([2, 3], [3]))       : rules[0],
    str(CellAuto([1, 5], [2, 3]))    : rules[1],
    str(CellAuto([4, 6], [1, 6]))    : rules[2]
}

A special case were rules are <i>{live: [2,3], dead: [3]}</i> is called <b>[Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)</b><br>

There are lot of repetitive patterns that the systen can converge into.<br>

![wikipedia1](https://upload.wikimedia.org/wikipedia/commons/0/07/Game_of_life_pulsar.gif)


In [28]:
video = Animator(CellAuto([2, 3], [3], distribution=[0.8, 0.2]))
HTML(video(path="images/cell_amim1.gif").to_html5_video())

<h2>Convolution model to predict `Conways Game of Life` next generation</h2>

In this section we will train a model to predict next generation.<br>
The data set contains single rule - <i>the conways game of life</i><br>

Please note that the 3 convolution layers consists of 3x3 filters gaining information of the direct neighbours<br>

However unlike typical image processing CNN we dont need any information <b>more than</b> direct neighbours.<br>

For that reason no down sampling is made except for the last fully connected layer that brings back to input size nedded for the <i>loss function</i><br>


In [30]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden1 = nn.Conv2d(1, 2, 3, stride=1, padding=3)
        self.hidden2 = nn.Conv2d(2, 4, 3, stride=1, padding=2) 
        self.hidden3 = nn.Conv2d(4, 8, 3, stride=1, padding=2) 
        
        self.fc1 = nn.Linear(8 * 24 * 24, input_size**2)
        self.sm = nn.Sigmoid()
        
    def forward(self, input_data):        
        x = F.relu(self.hidden1(input_data))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))

        x = torch.flatten(x, start_dim=1)        
        x = self.fc1(x)
        x = torch.reshape(x, (-1, 1, self.input_size, self.input_size))
        return self.sm(x)


![cell1](images/cell1.png)

<h3>Training the model</h3>

On Each epoch we will generate new data set randomly.<br>

On a 16x16 grid there is very small chance of <b>data repetitions</b><br>
Infact since the initial random settings of 1's and 0's at the begining of each game
is a [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution),<br> 
calculating its CDF reveals with confidence of 99% there will be <b>at least 11 1's</b> where the probability is 0.5.<br>
In that case the probability of initial data repetition is:
$$
\begin{equation*}
P(E)   = {256 \choose 11} ~= 6.23 * 10^{18}
\end{equation*}
$$

<b>The Loss Function</b> is standard [MSE](https://en.wikipedia.org/wiki/Mean_squared_error)
between the input (calculated next generation) and predicted (predicted next generation)

The predicted output is of continues float type while the calculated is of discrete zeros and ones<br>
This loss function is <b>over precise</b> to our needs as we need only minimalize <b>L(calculated - round(predicted))</b> however in our case it ensures faster converge<br>

In the <b>Validation function</b> we generate small data set and calculate the error percentage. In case its drop below 0.5% we finish our training.

In [77]:
N_GAMES_PER_EPOCH = 500
N_STATES_PER_GAME = 10
N_RULES           = 3
N_EPOCHS          = 1000
ROWS_PER_EPOCH    = 100
rules = torch.eye(N_RULES) 


def game_data(states=N_STATES_PER_GAME):
    arr = np.empty((states+1, 1, H, W), dtype=np.float32)
    auto_cell = CellAuto([2, 3], [3])       
    arr[0, 0, :, :] = auto_cell.data[:, :]
    
    for c in range(states):
        auto_cell.tick()
        arr[c+1, 0, :, :] = auto_cell.data[:, :]
    return arr

def get_data(games=N_GAMES_PER_EPOCH, states=N_STATES_PER_GAME):
    n_rows = games * states
    data   = np.empty((n_rows, 1, H, W), dtype=np.float32)
    target = np.empty((n_rows, 1, H, W), dtype=np.float32)
    
    c=0
    for i in range(games):
        tmp = game_data(states)
        for d, t in zip(tmp[: -1], tmp[1: ]):
            data[c, 0, :, :] = d[:, :, :]
            target[c, 0, :, :] = t[:, :, :]
            c+= 1
    return data, target
    
    
def test_net(model):
    arr, target = get_data(20, N_STATES_PER_GAME)
    res = model(torch.from_numpy(arr.copy())).detach().numpy()

    res[res>.5] = 1
    res[res<=.5] = 0
    
    res = res.astype(np.uint8)
    target = target.astype(np.uint8)
    n_errors = (res!=target).sum()
    total = target.shape[0] * W * H
    return n_errors / total * 100.



model = Net(W)
objective = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(params=model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=.5)

arr, target = get_data()
N_TEST_ROWS = arr.shape[0]


for i in range(N_EPOCHS):
    labels = target
    avg_loss = []
    n = 0
    while n < N_TEST_ROWS:
        X = torch.from_numpy(arr[n: n + ROWS_PER_EPOCH])
        Y = torch.from_numpy(labels[n: n + ROWS_PER_EPOCH])
        res    = model(X)
        
        loss_v = objective(res, Y)
        avg_loss.append(loss_v.detach().numpy())
        n += ROWS_PER_EPOCH
    
        optimizer.zero_grad()
        loss_v.backward()
        optimizer.step()
    scheduler.step()
        
    if i % 10 == 0:
        test_perc = test_net(model) 
        print(f"#{i} loss:{sum(avg_loss) / len(avg_loss)}, errors: {test_perc}%")
        
        if test_perc < .5:
            torch.save(model.state_dict(), "./model_gameoflife1")
            print("FINISH ALL")
            break
    arr, target = get_data()
print(f"#{i} loss:{loss_v}, errors: {test_perc}%")

#0 loss:4644.9196875, errors: 23.3125%
#10 loss:2625.8817724609376, errors: 12.958984374999998%
#20 loss:825.4164392089843, errors: 2.958984375%
#30 loss:150.0592790222168, errors: 0.24609375%
FINISH ALL
#30 loss:134.13668823242188, errors: 0.24609375%


In [34]:
model_gameoflife = Net(W)
model_gameoflife.load_state_dict(torch.load("./model_gameoflife1"))

video = Animator(CellAuto([2, 3], [3]), model=model_gameoflife)
HTML(video(path="images/cell_amim2.gif").to_html5_video())

<h2>Training model with multiple rules</h2>

Here we are about to upgrade the model to be able to predict [cellular automaton](https://en.wikipedia.org/wiki/Cellular_automaton) next generation with 3 distinct rules.<br>

Each rule applies totally distinct behavior from the model<br>

The model input will be the current generation matrix (16x16) and the rule discriminator vector (size 3)<br>

Please note how the rule vector merge itself into the first convolution layer as separate channel - I found it way much better in terms of accuracy and time convergence than any other configuration such as fully connected between or after the 3 convolution layers.

In [38]:
class Net(nn.Module):
    def __init__(self, input_size, rule_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden1 = nn.Conv2d(2, 4, 3, stride=1, padding=3, bias=True)
        self.hidden2 = nn.Conv2d(4, 8, 3, stride=1, padding=2, bias=True) 
        self.hidden3 = nn.Conv2d(8, 4, 3, stride=1, padding=2, bias=True) 
        
        self.fc_rule = nn.Linear(rule_size, input_size ** 2)
        self.fc1 = nn.Linear(4 * 24 * 24, input_size ** 2)
        self.sm = nn.Sigmoid()
        
    def forward(self, input_data, input_rule):
        rule_data = self.fc_rule(input_rule)
        rule_data = torch.reshape(rule_data, (-1, 1, self.input_size, self.input_size))
        
        mat = torch.cat((input_data, rule_data), 1)
        
        x = F.relu(self.hidden1(mat))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))

        x = torch.flatten(x, start_dim=1)        
        x = self.fc1(x)
        x = torch.reshape(x, (-1, 1, self.input_size, self.input_size))
        return self.sm(x)


![cell2](images/cell2.png)

In [9]:
N_GAMES_PER_EPOCH = 500
N_STATES_PER_GAME = 10
N_RULES           = 3
N_EPOCHS          = 1000
ROWS_PER_EPOCH    = 100
rules = torch.eye(N_RULES) 


def game_data(states=N_STATES_PER_GAME):
    arr = np.empty((states+1, 1, H, W), dtype=np.float32)
    mode = np.random.randint(N_RULES)
    if mode == 0:
        auto_cell = CellAuto([2, 3], [3])
    elif mode == 1:
        auto_cell = CellAuto([1, 5], [2, 3])
    elif mode == 2:
        auto_cell = CellAuto([4, 6], [1, 6])
        
    arr[0, 0, :, :] = auto_cell.data[:, :]
    
    for c in range(states):
        auto_cell.tick()
        arr[c+1, 0, :, :] = auto_cell.data[:, :]
    rule = torch.zeros((states, N_RULES), dtype=torch.float32)
    rule[:, mode] = 1.
    return arr, rule

def get_data(games=N_GAMES_PER_EPOCH, states=N_STATES_PER_GAME):
    n_rows = games * states
    data   = np.empty((n_rows, 1, H, W), dtype=np.float32)
    target = np.empty((n_rows, 1, H, W), dtype=np.float32)
    rule_res = torch.zeros((n_rows, N_RULES), dtype=torch.float32)
    
    c=0
    for i in range(games):
        tmp, rule = game_data(states)
        rule_res[c: c+N_STATES_PER_GAME, :] = rule
        for d, t in zip(tmp[: -1], tmp[1: ]):
            data[c, 0, :, :] = d[:, :, :]
            target[c, 0, :, :] = t[:, :, :]
            c+= 1
    return data, target, rule_res
    
    
def test_net(model):
    arr, target, rule = get_data(10, N_STATES_PER_GAME)
    res = model(torch.from_numpy(arr.copy()), rule).detach().numpy()

    res[res>.5] = 1
    res[res<=.5] = 0
    target[target>.5] = 1
    target[target<=.5] = 0
    
    res = res.astype(np.uint8)
    target = target.astype(np.uint8)
    n_errors = (res!=target).sum()
    total = target.shape[0] * W * H
    return n_errors / total * 100.



model = Net(W, N_RULES)
objective = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(params=model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=.5)

arr, target, rule = get_data()
N_TEST_ROWS = arr.shape[0]


for i in range(N_EPOCHS):
    labels = target
    avg_loss = []
    n = 0
    while n < N_TEST_ROWS:
        X = torch.from_numpy(arr[n: n + ROWS_PER_EPOCH])
        Y = torch.from_numpy(labels[n: n + ROWS_PER_EPOCH])
        res    = model(X, rule[n: n + ROWS_PER_EPOCH])
        
        loss_v = objective(res, Y)
        avg_loss.append(loss_v.detach().numpy())
        n += ROWS_PER_EPOCH
    
        optimizer.zero_grad()
        loss_v.backward()
        optimizer.step()
    scheduler.step()
        
    if i % 10 == 0:
        test_perc = test_net(model) 
        print(f"#{i} loss:{sum(avg_loss) / len(avg_loss)}, errors: {test_perc}%")
        
        if test_perc < .5:
            torch.save(model.state_dict(), "./model_gameoflife2")
            print("FINISH ALL")
            break
    arr, target, rule = get_data()
print(f"#{i} loss:{loss_v}, errors: {test_perc}%")

#0 loss:5371.42984375, errors: 27.60546875%
#10 loss:4221.880561523438, errors: 23.37109375%
#20 loss:2179.538444824219, errors: 10.0390625%
#30 loss:1457.9715478515625, errors: 6.94140625%
#40 loss:1051.339052734375, errors: 6.015625%
#50 loss:824.8548754882812, errors: 3.94921875%
#60 loss:705.9457312011718, errors: 3.84375%
#70 loss:610.3816284179687, errors: 3.03515625%
#80 loss:541.9419702148438, errors: 2.96484375%
#90 loss:499.49175354003904, errors: 1.9921874999999998%
#100 loss:469.44502990722657, errors: 2.60546875%
#110 loss:427.9045489501953, errors: 2.71875%
#120 loss:426.7676690673828, errors: 1.2421875%
#130 loss:377.80281707763675, errors: 1.515625%
#140 loss:376.5887103271484, errors: 1.65625%
#150 loss:359.849499206543, errors: 1.25390625%
#160 loss:322.77151275634765, errors: 2.3359375%
#170 loss:344.4906951904297, errors: 2.04296875%
#180 loss:336.3598794555664, errors: 0.8593750000000001%
#190 loss:303.2706579589844, errors: 1.94140625%
#200 loss:314.4601712036133,

In [43]:
model_gameoflife = Net(W, N_RULES)
model_gameoflife.load_state_dict(torch.load("./model_gameoflife2"))

video = Animator(CellAuto([4, 6], [1, 6]), model=model_gameoflife)
HTML(video(path="images/cell_amim3.gif").to_html5_video())

<h2>Reverse Engineering Cellular Automaton</h2>

In this section we will train a model to classify the exact rule that produce 2 consequent generations.<br>
This time the input will be 2 16x16 matrices representing the 2 consequent generations and the output vector of size 3 representing the rule class.<br>

Each input matrix assemble to separate channel upsampled and then down sampled<br>
Finally there are 2 fully connected layers to reach its final size

In [89]:
class Net(nn.Module):
    def __init__(self, input_size, out_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden1 = nn.Conv2d(2, 4, 3, stride=1, padding=3, bias=True)
        self.hidden2 = nn.Conv2d(4, 8, 3, stride=1, padding=2, bias=True) 
        self.hidden3 = nn.Conv2d(8, 4, 3, stride=1, padding=2, bias=True) 
        
        self.fc1 = nn.Linear(4 * 24 * 24, 128)
        self.fc2 = nn.Linear(128, out_size)
        self.sm = nn.Sigmoid()
        
    def forward(self, input_data1, input_data2):
        mat = torch.cat((input_data1, input_data2), 1)
        
        x = F.relu(self.hidden1(mat))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))

        x = torch.flatten(x, start_dim=1)        
        x = self.fc1(x)
        x = self.fc2(x)
        return self.sm(x)


![title](images/cell3.png)

In [51]:
N_GAMES_PER_EPOCH = 500
N_STATES_PER_GAME = 10
N_EPOCHS          = 1000
ROWS_PER_EPOCH    = 100


def game_data(states=N_STATES_PER_GAME):
    arr = np.empty((states+1, 1, H, W), dtype=np.float32)
    mode = np.random.randint(N_RULES)
    if mode == 0:
        auto_cell = CellAuto([2, 3], [3])
    elif mode == 1:
        auto_cell = CellAuto([1, 5], [2, 3])
    elif mode == 2:
        auto_cell = CellAuto([4, 6], [1, 6])
        
    arr[0, 0, :, :] = auto_cell.data[:, :]
    
    for c in range(states):
        auto_cell.tick()
        arr[c+1, 0, :, :] = auto_cell.data[:, :]
    rule = torch.zeros((states, N_RULES), dtype=torch.float32)
    rule[:, mode] = 1.
    return arr, rule

def get_data(games=N_GAMES_PER_EPOCH, states=N_STATES_PER_GAME):
    n_rows = games * states
    data1  = np.empty((n_rows, 1, H, W), dtype=np.float32)
    data2  = np.empty((n_rows, 1, H, W), dtype=np.float32)
    target = torch.zeros((n_rows, N_RULES), dtype=torch.float32)
    
    c=0
    for i in range(games):
        tmp, rule = game_data(states)
        target[c: c+N_STATES_PER_GAME, :] = rule
        for d, t in zip(tmp[: -1], tmp[1: ]):
            data1[c, 0, :, :] = d[:, :, :]
            data2[c, 0, :, :] = t[:, :, :]
            c+= 1
    return data1, data2, target
    
    
def test_net(model):
    arr1, arr2, target = get_data(20, N_STATES_PER_GAME)
    res = model(torch.from_numpy(arr1), torch.from_numpy(arr2)).detach().numpy()

    res[res>.5] = 1
    res[res<=.5] = 0
    target[target>.5] = 1
    target[target<=.5] = 0
    
    res = res.astype(np.uint8)
    target = target.detach().numpy().astype(np.uint8)
    n_errors = (res!=target).sum()
    total = target.shape[0] * W * H
    return n_errors / total * 100.



model = Net(W, N_RULES)
objective = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(params=model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=.5)

arr1, arr2, labels = get_data()
N_TEST_ROWS = arr.shape[0]


for i in range(N_EPOCHS):
    avg_loss = []
    n = 0
    while n < N_TEST_ROWS:
        X1 = torch.from_numpy(arr1[n: n + ROWS_PER_EPOCH])
        X2 = torch.from_numpy(arr2[n: n + ROWS_PER_EPOCH])
        Y  = labels[n: n + ROWS_PER_EPOCH]
        res    = model(X1, X2)
        
        loss_v = objective(res, Y)
        avg_loss.append(loss_v.detach().numpy())
        n += ROWS_PER_EPOCH
    
        optimizer.zero_grad()
        loss_v.backward()
        optimizer.step()
    scheduler.step()
        
    if i % 5 == 0:
        test_perc = test_net(model) 
        print(f"#{i} loss:{sum(avg_loss) / len(avg_loss)}, errors: {test_perc}%")
        
        if test_perc < .05 and avg_loss[-1] < 1.:
            torch.save(model.state_dict(), "./model_gameoflife3")
            print("FINISH ALL")
            break
    arr1, arr2, labels = get_data()


#0 loss:60.44437698364258, errors: 0.244140625%
#5 loss:7.767050590515137, errors: 0.033203125%
#10 loss:3.2951562225818636, errors: 0.0078125%
#15 loss:1.6691913002729415, errors: 0.0078125%
FINISH ALL


### Correctness testing

In [72]:
auto_cells = [ 
              CellAuto([2, 3], [3]), 
              CellAuto([1, 5], [2, 3]), 
              CellAuto([4, 6], [1, 6])
            ]

for cell in auto_cells:
    expected = RULES_MAP[str(cell)]
    last = cell.data.copy().astype(np.float32).reshape((1, 1, H, H))
    for i in range(10):
        cell.tick()
        curr = cell.data.copy().astype(np.float32).reshape((1, 1, H, H))
        res = model(torch.from_numpy(last), torch.from_numpy(curr))
        pred = res.flatten().round()

        assert (pred == expected).all(), "Incorrect" 
        last[:,:,:,:] = curr[:,:,:,:]